# Benji, Wendi, Yiwen

Dataset can be downloaded at:

https://www.kaggle.com/zynicide/wine-reviews

In [27]:
install.packages("glmnet")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [28]:
library(readr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(gbm)
library(gam)
library(stringr)
library(Matrix)
library(glmnet)

Loaded glmnet 2.0-16



Read in the data, drop duplicates, drop `description`, `winery`, and `taster_twitter_handle`.

In [2]:
read_csv("Data/wine-reviews/winemag-data-130k-v2.csv") %>% select(-X1) %>% unique %>% 
select(-description, -winery, -taster_twitter_handle) -> data

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_double(),
  country = col_character(),
  description = col_character(),
  designation = col_character(),
  points = col_double(),
  price = col_double(),
  province = col_character(),
  region_1 = col_character(),
  region_2 = col_character(),
  taster_name = col_character(),
  taster_twitter_handle = col_character(),
  title = col_character(),
  variety = col_character(),
  winery = col_character()
)


In this notebook, we will attempt to automate some of the data munging/imputation. 

We want to scale `points` by the taster, since the distribution of points is likely dependent on the taster. Therefore, we will attempt two different methods of unifying this distribution: scaling by standardization and scaling by percentile of score.

In [3]:
scale_taster <- function(points){
    # takes a vector of numbers, subtracts every element by the mean of the vector, and then
    # divides every element by the standard deviation of the vector
    
    return((points - mean(points, na.rm = TRUE)) / sd(points, na.rm = TRUE))
}

In [4]:
percentile_taster <- function(x){
    # takes a vector of numbers, ranks every element and divides by n, giving the percentile of each element
    trunc(rank(x))/length(x) * 100
}

In [5]:
data <- data %>% group_by(taster_name) %>% mutate("Scaled Points" = scale_taster(points))

In [6]:
data <- data %>% group_by(taster_name) %>% mutate("Percentile Points" = percentile_taster(points))

We want to reduce the number of provinces. If a given province only appears less than 1% of the time, we want to change that province to its country instead. However, we want to distinguish these miscellaneous provinces from the major ones, so we will call it `country_other`.

In [7]:
tab <- data %>% group_by(province) %>% summarize("Proportion" = n()/nrow(data))
tab <- tab[tab$Proportion > 0.01, ]

tabcountry <-  data %>% group_by(country) %>% summarize("Proportion" = n()/nrow(data))
tabcountry <- tabcountry[tabcountry$Proportion > 0.01, ]

In [8]:
data$country_other <- ifelse(data$country %in% tabcountry$country, 
                                paste0(data$country, "_other"), data$country)
data$location <- ifelse(data$province %in% tab$province, data$province,
                                     data$country_other)

Now that we have consolidated provinces under the `location` variable, we can drop `province` and `country_other` since we no longer have any need for them.

In [9]:
data <- data %>% select(-province, -country_other)

In [10]:
year <- str_extract_all(data$title, "[1-2][09][0-9]{2}")

In [11]:
data$year <- lapply(year, function(x){
    x = x %>% as.numeric
    if(!all(is.na(x))){
        #newx <- max(x[(x > 1900) & (x < 2018)])
        #return(newx)
        return(max(x))
    }
    else {
        return(NA) 
    }}) %>% unlist

In [12]:
summary(data$year)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1904    2009    2011    2011    2013    2900    4285 

In [155]:
data1 <- data
data1[which(is.na(data1$taster_name)), "taster_name"] <- "Missing"
data1$taster_name <- addNA(data1$taster_name)
data1$location <- addNA(data1$location)

data1 %>% group_by(variety) %>% summarize("Average_Price" = mean(price, na.rm = T),"Count" = n()) %>% 
    filter(is.na(Average_Price) | Count < 50) %>% select(variety) %>% unlist() -> drop_var

data1 <- data1[-which(data1$variety%in%drop_var),]

data1$variety <- addNA(data1$variety)

In [156]:
dim(data1)

[1] 115094     13

We want to impute `price` by the average price of the missing wine's variety, i.e. we will group by variety, take the mean, and then use that value to impute. However, we won't be able to impute the price of wines where the variety has only missing prices because then the mean wouldn't be defined. Thus, these cases will be removed.

In [15]:
impute_mean <- function(x) replace(x, is.na(x), mean(x, na.rm = TRUE))
    # impute_mean replaces missing values with the average value of a group

In [112]:
clean <- function(df){
    # clean removes the varieties that only have missing prices, and are thus unimputable by our rule,
    # and then it imputes the remaining missing prices using the average price of that wine's variety
    
    df %>% group_by(variety) %>% summarize("Average_Price" = mean(price, na.rm = T), 
                                           "Count" = n()) %>% 
    filter(is.na(Average_Price)) %>% select(variety) %>% unlist() -> drop_variety 
    
    df %>% filter(!(variety %in% drop_variety)) -> sample2
    
    return(sample2 %>% group_by(variety) %>% mutate(price = impute_mean(price)))
}

For lasso/ridge model: need to convert to sparse matrix

In [161]:
set.seed(2018)
k <- 10
sp <- split(c(1:nrow(data1)),c(1:k))

Warning message in split.default(c(1:nrow(data1)), c(1:k)):
“data length is not a multiple of split variable”

In [177]:
lasso.err <- c()
ridge.err <- c()

In [179]:
for(i in 1:k){
    train <- data1[-sp[[k]], ]
    test <- data1[sp[[k]], ]
    
    cleanwine_train <- clean(train)
    cleanwine_test <- clean(test)

    cleanwine_train <- cleanwine_train %>% select(points,price,taster_name,variety,location)
    cleanwine_test <- cleanwine_test %>% select(points,price,taster_name,variety,location)
    
    trn.mtx <- model.matrix(~.,cleanwine_train)
    trn.smtx <- Matrix(trn.mtx,sparse=T)[,-1]
    
    tst.mtx <- model.matrix(~.,cleanwine_test)
    tst.smtx <- Matrix(tst.mtx,sparse=T)[,-1]
    
    fit.lasso <- cv.glmnet(x=trn.smtx,cleanwine_train$price,alpha = 1,type.measure = "mae") 
    l.err <- predict(fit.lasso,newx = tst.smtx,type = 'response')
    lasso.err <- c(lasso.err,(l.err-cleanwine_test$price)^2)
 
    fit.ridge <- cv.glmnet(x=trn.smtx,cleanwine_train$price,alpha=0,type.measure = "mae")
    r.err <- predict(fit.ridge,newx = tst.smtx, type = 'response')
    ridge.err <- c(ridge.err,(r.err-cleanwine_test$price)^2)
}  
mean(lasso.err)
mean(ridge.err)

[1] 18.53105

[1] 24.13624